In [1]:
from alpaca_farm.models import reward_model
from alpaca_farm.inference.decode import load_model_and_tokenizer_for_inference 
from alpaca_farm.inference.score import score_sequences_with_huggingface_given_model
from transformers import T5Tokenizer, T5ForConditionalGeneration
from alpaca_farm import utils
import pandas as pd
from statistics import mean, stdev
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from src.utils.samp_utils import gen_row, get_reward_single, get_reward_double
import alpaca_farm.data_preprocessor as data_preprocessor
import random

device="cuda:1"

/home/prasann/miniconda3/envs/apfarmenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
steamtok = T5Tokenizer.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl')
steamshp = T5ForConditionalGeneration.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl').to(device)
steamshp.eval()

In [2]:
model, tokenizer = load_model_and_tokenizer_for_inference(
    model_name_or_path="/home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/",
    model_cls=reward_model.RewardModel,
    cache_dir=None,
    model_kwargs=dict(
        torch_dtype=utils.convert_str_dtype_to_torch_dtype(None),
        flash_attn=False,
    ),
)

Loading model for inference: /home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:29<00:00, 49.68s/it]


In [ ]:
model, tokenizer = load_model_and_tokenizer_for_inference(
    model_name_or_path='/home/prasann/Projects/tfr-decoding/apfarm_models/ppo-human/',
    cache_dir=None,
    model_kwargs=dict(torch_dtype=utils.convert_str_dtype_to_torch_dtype(None)),
)

In [ ]:
def generate_stuff(question):
    pdict_path = "/home/prasann/Projects/tfr-decoding/alpaca_farm/examples/prompts/v0_inputs_noinputs.json"
    prompts, list_dict_data, metadata = data_preprocessor.format_prompt_with_data_frame(
        df=pd.DataFrame({'instruction':[question], 'input':[""]}),
        prompt_dict=utils.jload(pdict_path),
    )
    inps = tokenizer(prompts, return_tensors="pt").to("cuda")
    outs = model.generate(inps.input_ids, attention_mask=inps.attention_mask, do_sample=True, temperature=1.0,max_new_tokens=300,num_return_sequences=2,)
    return tokenizer.batch_decode(outs, skip_special_tokens=True)

In [ ]:
otmp = generate_stuff(alldfs['instruction'].loc[0])

In [ ]:
scos = [float(get_reward_single({"context": alldfs['instruction'].loc[0], "hyp":o}, steamtok, steamshp)) for o in otmp]
print(scos)

In [ ]:
print(otmp[1])

In [ ]:
ppoouts = pd.read_json("alpaca_farm/bon_outs/ppohum.jsonl", orient="records").drop(columns=['input', 'scorer_name_or_path'])
sftouts = pd.read_json("alpaca_farm/bon_outs/sft.jsonl", orient="records").drop(columns=['input', 'scorer_name_or_path'])


In [ ]:
def scoreseq(inp, ans):
    # TODO make a batched version if necessary
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:{response}"
    seq = template.format(instruction=inp, response=ans)
    return score_sequences_with_huggingface_given_model(model, tokenizer, [seq], per_device_batch_size=1)
    

In [ ]:
inp = ppoouts.loc[1]['instruction']
hyp = ppoouts.loc[1]['output'][1]
scoreseq(inp, hyp)

In [ ]:
ppoouts.loc[1].top_index[0]

In [ ]:
def printex(row):
    print(row.instruction)
    for i in [1, 2, 6, 7]:
        topind = row.top_index[i]
        print(row.top_scores[topind])
        print(row.output[topind])

In [ ]:

ind = 15
print("_________")
print("BEFORE PPO")
printex(sftouts.loc[ind])
print("_________")
print("AFTER PPO")
printex(ppoouts.loc[ind])


In [ ]:
mean([mean(m) for m in ppoouts.top_scores])

In [ ]:
lendist = []
for sc in sftouts.output:
    lendist.extend([len(tokenizer(s).input_ids) for s in sc])
print(mean(lendist))
plt.hist(lendist)

In [ ]:
scodist = []
for sc in sftouts.top_scores:
    scodist.extend(sc)
plt.hist(scodist)

In [ ]:
nldist = []
nsdist = []
for i in range(len(lendist)):
    if lendist[i]>=300:
        continue
    nldist.append(lendist[i])
    nsdist.append(scodist[i])

In [ ]:
spearmanr(nldist, nsdist)

In [ ]:
plt.scatter(nldist, nsdist)

In [ ]:
sftouts.loc[7]['output']

In [2]:
from os import listdir

In [3]:
dfs = []
for dir in listdir("alpaca_farm/bon_outs/"):
    if 'eli' in dir:
        dfs.append(pd.read_json("alpaca_farm/bon_outs/"+dir, orient='records'))

In [5]:
ppo_diag = pd.read_json("alpaca_farm/bon_outs/ppohum_diag.jsonl", orient='records')
sft_diag = pd.read_json("alpaca_farm/bon_outs/sft_diag.jsonl", orient='records')

In [16]:
sft_diag.to_json("alpaca_farm/bon_outs/sft_diag.jsonl", orient='records')

In [4]:
alldfs = pd.concat(dfs).reset_index(drop=True).drop(columns=['input', 'scorer_name_or_path'])

In [5]:
alldfs

,instruction,output,top_sequence,top_index,top_scores
0,"Gorillaz and their virtual band ""history""I am ...",[Gorillaz is a British virtual band created in...,[Below is an instruction that describes a task...,"[1, 0]","[0.39429283142089805, 0.7290570735931391]"
1,If a RAR or ZIP file just make another file/gr...,"[Yes, you can keep compressing RAR and ZIP fil...",[Below is an instruction that describes a task...,"[1, 0]","[1.037175416946411, 1.441125512123108]"
2,Why do Catalonia and the rest of Spain think e...,[The conflict between Catalonia and the rest o...,[Below is an instruction that describes a task...,"[0, 1]","[1.635269045829773, 0.12795141339302002]"
3,Who chooses where the Olympics are going to be...,[The International Olympic Committee (IOC) is ...,[Below is an instruction that describes a task...,"[1, 0]","[-0.598800957202911, 2.12459659576416]"
4,How could the governments of USA / UK have giv...,[It is understandable that you may be confused...,[Below is an instruction that describes a task...,"[0, 1]","[0.7304080724716181, 0.454102754592895]"
...,...,...,...,...,...
16695,Pangaea: Why was there just a single giant lan...,[The hypothesis of a single super-continent is...,[Below is an instruction that describes a task...,"[0, 1]","[2.8094208240509033, 2.256419658660888]"
16696,"All the different FPS ""netcode"" terminology li...",[Netcode is a term used to refer to the networ...,[Below is an instruction that describes a task...,"[0, 1]","[1.205457210540771, 0.6027202010154721]"
16697,Where do last names come from?There's a few qu...,[Last names are thought to have originated in ...,[Below is an instruction that describes a task...,"[1, 0]","[1.125969529151916, 1.569443464279174]"
16698,"Why is the skin on penises so dark, compared t...",[The skin on penises is darker than other area...,[Below is an instruction that describes a task...,"[0, 1]","[1.840420961380004, 1.813018083572387]"


In [ ]:
alldfs

In [38]:
def scoind(ind, adf):
    row = adf.loc[ind]
    scos = [float(get_reward_single({"context": row['instruction'], "hyp":o}, steamtok, steamshp)) for o in row['output']]
    return scos

def budgind(ind, adf):
    row = adf.loc[ind]
    budgs = [len(preftok(o).input_ids) for o in row['output']]
    return budgs

def adaptbase(inpdf, thresh, mrange):
    inds = list(range(8))
    fscos = []
    budgets = []
    for ind in range(len(inpdf)):
        slist = inpdf.shpscos[ind]
        blist = inpdf.budgs[ind]
        random.shuffle(inds)
        tmp = []
        budget = 0
        # get adaptive score
        for i in inds[:mrange]:
            budget = budget+blist[i]
            if slist[i]>thresh:
                fscos.append(slist[i])
                break
            tmp.append(slist[i])
            if i==inds[mrange-1]:
                fscos.append(max(tmp))
        budgets.append(budget)
    return mean(fscos), mean(budgets)

def adaptive_prefsort(inpdf, thresh, mrange, pf, hstop):
    inds = list(range(8))
    fscos = []
    budgets = []
    for ind in range(len(inpdf)):
        # score list and list of budgets
        slist = inpdf.shpscos[ind]
        blist = inpdf.budgs[ind]
        # list with prefix metric scores (specifically final class)
        plist = [a[-1] for a in inpdf["probs"+str(pf)][ind]]
        # mix up what order we get stuff in
        random.shuffle(inds)
        tmp = []
        # get prob values
        nplist = [plist[p] for p in inds[:mrange]]
        
        # get indices to use
        sortps = [inds[pl] for pl in np.argsort(nplist)]
        sortps.reverse()
        # if ind==0:
        #     print(plist)
        #     print(slist)
        #     print(sortps)
        budget = pf*mrange
        # get adaptive score
        for i in sortps[:hstop]:
            budget = budget+blist[i]
            budget = budget-pf # already part of budget, remove
            if slist[i]>thresh:
                fscos.append(slist[i])
                break
            tmp.append(slist[i])
            if i==sortps[hstop-1]:
                fscos.append(max(tmp))
        budgets.append(budget)
    return mean(fscos), mean(budgets)

In [39]:
for j in range(1, 9):
    scos, buds = [], []
    for i in range(500):
        s, b = adaptive_prefsort(sft_diag, .94, 8, 20, j)
        # s, b = samprer(fulld,  j)
        scos.append(s)
        buds.append(b)
    print(mean(scos), " ", mean(buds))

0.9100416481494904   260.28
0.9312801229953765   314.22
0.9333545243740082   350.52
0.9348818624019622   385.82
0.9363791239261627   419.74
0.9372864806652069   451.46
0.9381633234024048   486.02
0.9395368981361389   515.96


In [ ]:
ascos = []
for i in range(50):
    print(i)
    ascos.append(scoind(i, sft_diag))

In [11]:
sft_diag['shpscos'] = ascos

In [14]:
sft_diag['budgs'] = [budgind(i, sft_diag) for i in range(len(sft_diag))]

In [ ]:
mean([mean(m) for m in sft_diag['budgs']])

In [30]:
def make_prefs(indf, tok, pflen):
    res = []
    for h in indf.output:
        prefs = [tok.decode(tok(hyp).input_ids[:pflen], skip_special_tokens=True) for hyp in h]
        res.append(prefs)
    return res
CLASSES = [71, 272, 205, 309]
def getprobs(indf, ex):
    allprobs = []
    for num in range(8):
        out = qpref.predsingle(indf['instruction'][ex], indf['pf20'][ex][num], True)
        cprobs = []
        for c in CLASSES:
            out.sequences[0][1] = c
            transition_scores = qpref.model.compute_transition_scores(
                out.sequences, out.scores, normalize_logits=True
            )
            cprobs.append(float(np.exp(transition_scores[0][0].cpu())))
        allprobs.append(cprobs)
    return allprobs

In [32]:
import numpy as np
import torch
torch.cuda.empty_cache()

In [4]:
from src.tfr_decoding.shp_modeling import T5BinaryClassifier

pfmod_path = "./lightning_logs/multimodel/checkpoints/epoch=3-step=86946.ckpt"
pfname = 'stanfordnlp/SteamSHP-flan-t5-large'
# get prefix model
qpref = T5BinaryClassifier.load_from_checkpoint(pfmod_path, device_map="auto")
preftok = T5Tokenizer.from_pretrained(pfname)

In [21]:
sft_diag['pf20'] = make_prefs(sft_diag, preftok, 20)

In [33]:
with torch.no_grad():
    allprobs = []
    for val in range(len(sft_diag)):
        if val%10==0:
            print(val)
        allprobs.append(getprobs(sft_diag, val))

0
10
20
30
40


In [34]:
sft_diag['probs20'] = allprobs

In [ ]:
# get dataframes with prefix scores for different outs that we'll use in reranking
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

In [ ]:
sft_diag